In [1]:
# Vamos a utilizar el data set de MNIST

In [2]:
import torch
# Modulo de NN
import torch.nn as nn 
# Modulo para los algoritmos de optimizacion
import torch.optim as optim
# Funciones de activacion
import torch.nn.functional as F
# Este modulo nos permite manipular datos 
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
# Creamos la clase que define a la arquitectura que vamos a utilizar
class NN(nn.Module):
    
    # MNIST tiene 10 clases, números del 0 al 9
    def __init__(self, tam_entrada, num_clases): 

        super(NN, self).__init__()

        # Como las imágenes van a ser de 28 x 28 = 784 pixeles por imagen, este será  el tamaño de entrada
        # el tamaño de la primera capa
        self.capa1 = nn.Linear(in_features = tam_entrada, out_features = 50)
        self.capa2 = nn.Linear(in_features = 50, out_features = num_clases)

    def forward(self, entrada):

        entrada = F.relu(self.capa1(entrada))
        entrada = self.capa2(entrada)
        
        return entrada


In [6]:
# Podemos hacer una prueba de que la estructura es correcta si al darle un tensor cualquiera obtenermos un resultado con un tamño esperado
modelo = NN(tam_entrada = 784, num_clases = 10)

# 64 sería el número de ejemplos que ejecutaríamos de manera simultanea, es decir, el batch size
x = torch.rand(size = (64, 784))

# Observaremos que el resultado será un tensor de tamaño igual a la cantidad de ejemplos del batch size y el número de clases
print(modelo(x).shape)

torch.Size([64, 10])


In [7]:
# Hiperparámetros
tam_entrada = 784
num_clases = 10
batch_size = 64
learning_rate = 1e-3
num_epocas = 3

In [8]:
# Vamos a descargar el dataset de MNIST
train_dataset = datasets.MNIST(root = 'dataset/', train = True, transform = transforms.ToTensor(), download = True)
train_loader = DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True)

test_dataset = datasets.MNIST(root = 'dataset/', train = False, transform = transforms.ToTensor(), download = True)
test_loader = DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle = True)

100.1%

Extracting dataset/MNIST\raw\train-images-idx3-ubyte.gz to dataset/MNIST\raw


113.5%

Extracting dataset/MNIST\raw\train-labels-idx1-ubyte.gz to dataset/MNIST\raw


100.4%

Extracting dataset/MNIST\raw\t10k-images-idx3-ubyte.gz to dataset/MNIST\raw


180.4%e:\Programas\anaconda3\envs\pytorch_1.7.1\lib\site-packages\torchvision\datasets\mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Extracting dataset/MNIST\raw\t10k-labels-idx1-ubyte.gz to dataset/MNIST\raw
Processing...
Done!


In [9]:
# Inicializamos el modelo
# Usamos .to(device) para llevar el entrenamiento del modelo al dispositivo disponible (GPU o CPU)
modelo = NN(tam_entrada = tam_entrada, num_clases = num_clases).to(device)

In [10]:
# Loss function, para clasificaciones de una sola etiqueta (y) como en este caso podemos utilizar Cross Entropy Loss
func_perdida = nn.CrossEntropyLoss()
optimizador = optim.Adam(params = modelo.parameters(), lr = learning_rate)

In [12]:
# Variable utilizada para controlar el tamaño de los datos
cont = 0

for epoca in range(num_epocas):

    for batch_index, (x, y) in enumerate(iterable = train_loader):

        x = x.to(device = device)
        y = y.to(device = device)

        if (cont == 0):

            cont += 1

            # El resultado debe ser [batch_size, nº canales, tamaño imagen]
            # En este caso batch_size = 64, nº canales de la imagen es 1 porque son imagenes en blanco y negro
            # por lo que no tiene canales RGB, y el tamaño de cada imagen es de 28 x 28
            print(x.shape)

        # Pero tenemos que convertir estas matrices (una imagen es una matriz) en tensores de una dimension
        x = x.reshape(x.shape[0], -1)

        # forward
        scores = modelo(x)
        loss = func_perdida(scores, y)

        # Backpropagation
        # Zero your gradients for every batch!
        optimizador.zero_grad() # Ponemos el gradiente a 0 para cada batch para que no almacene las derivadas de las anteriores capas
        loss.backward()

        # Descenso del gradiente o pasos del algoritmo ADAM
        optimizador.step()

torch.Size([64, 1, 28, 28])


In [19]:
# Comprobar precisión del modelo en entrenamiento y validacion
def comprobar_precision(loader, model):

    num_correct = 0
    num_samples = 0
    model.eval()

    # Para evaluar el modelo no necesitamos calcular el gradiente
    with torch.no_grad(): 
    
        for x, y in loader:
    
            x = x.to(device = device)
            y = y.to(device = device)
            x = x.reshape(x.shape[0], -1)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(f"Tenemos {num_correct} / {num_samples} con una precision de {float(num_correct) / float(num_samples) * 100:.2f}")

    model.train()

In [20]:
comprobar_precision(train_loader, modelo)
comprobar_precision(test_loader, modelo)

Tenemos 57614 / 60000 con una precision de 96.02
Tenemos 9556 / 10000 con una precision de 95.56
